In [13]:
import numpy as np
from pynq import allocate
from pynq import MMIO
from pynq import Overlay
from enum import IntEnum
import time

overlay = Overlay("./design_1.bit")

cdma = overlay.axi_cdma_0
input_buffer = allocate(shape=(1024,), dtype=np.uint32)
output_buffer = allocate(shape=(1024,), dtype=np.uint32)
for i in range(1024):
   input_buffer[i] = i

print("Input:")

for i in range(5):
    print(str(input_buffer[i]) + ", ", end="")

print()

def getbit(value, order):
    orderVal = 2**order
    tmpbin = value & orderVal
    if (tmpbin):
        return 1
    else:
        return 0

def changebit(value, order, bit):
    if (bit == 1):
        return value | (1 << order)
    else:
        return value & ~(1 << order)

class Cdma(IntEnum):
    CDMACR = 0x0
    CDMASR = 0x4
    SA = 0x18
    DA = 0x20
    BTT = 0x28

def transfer(cdma, src, dst, size):
    # Step 1
    cdmasr = cdma.read(Cdma.CDMASR)
    cdmasrIdle = getbit(cdmasr, 1)
    if (cdmasrIdle != 1):
        print("CDMA is busy..")
        return

    # Step 2
    cdmacr = cdma.read(Cdma.CDMACR)
    cdmacr = changebit(cdmacr, 12, 1) # set IOC_IrqEn
    cdmacr = changebit(cdmacr, 14, 1) # set ERR_IrqEn
    cdma.write(Cdma.CDMACR, cdmacr)

    # Step 3
    cdma.write(Cdma.SA, src)

    # Step 4
    cdma.write(Cdma.DA, dst)

    # Step 5
    cdma.write(Cdma.BTT, size)

    # Step 6
    print("Transferring...")
    cdma.read(Cdma.CDMASR)
    cdmasrIdle = getbit(cdmasr, 1)
    while (cdmasrIdle != 1):
        print(".", end="")
        cdma.read(Cdma.CDMASR)
        cdmasrIdle = getbit(cdmasr, 1)

    # Step 7-8
    cdmasr = cdma.read(Cdma.CDMASR)
    cdmasr = changebit(cdmasr, 12, 1) # clear IOC_Irq
    cdma.write(Cdma.CDMASR, cdmasr)

    print("Transfered " + str(size) + " bytes from " + str(src) + " to " + str(dst))
    print("CDMA Done.")

def reset(cdma):
    cdmacr = cdma.read(Cdma.CDMACR)
    print(type(cdmacr))
    print(cdmacr)
    cdmacr = changebit(cdmacr, 2, 1)
    print(cdmacr)
    cdma.write(Cdma.CDMACR, cdmacr)

print("==== Test CDMA ====")

start_time = time.time()
transfer(cdma, input_buffer.physical_address, 0xE000_0000, 1024*4)
transfer(cdma, 0xE000_0000, output_buffer.physical_address, 1024*4)
end_time = time.time()

print("%s seconds" % (end_time - start_time))
print()
print(output_buffer)


Input:
0, 1, 2, 3, 4, 
==== Test CDMA ====
Transferring...
Transfered 256 bytes from 634380288 to 3758096640
CDMA Done.
Transferring...
Transfered 256 bytes from 3758096640 to 448798720
CDMA Done.
0.002915620803833008 seconds

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [3]:
overlay.axi_bram_ctrl_0?

Object `overlay.axi_bram_ctrl_0` not found.


In [4]:
overlay.axi_cdma_0